In [1]:
cd ..

d:\FarmconProject\plantClassificationPart


d:\FarmconProject\plantClassificationPart\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import tensorflow as tf

In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
# from tensorflow.keras.models.

In [4]:
FOLDER_PATH = os.path.join("artifacts", "data_ingestion", "plantData")
# for fPath in os.listdir(FOLDER_PATH):
#     imagePath = os.path.join(FOLDER_PATH, fPath)
#     for imgPath in os.listdir(imagePath):
#         image = plt.imread(os.path.join(imagePath, imgPath))
#         plt.imshow(image)
#         plt.show()
#         break

In [5]:
AUGMENTATION = True
IMAGE_SIZE = [224, 224, 3] # as per VGG 16 model
TARGET_SIZE = [224, 224]
BS = 16
INCLUDE_TOP = False
EPOCHS = 5
CLASSES = 88
# WEIGHTS: imagenet
LEARNING_RATE = 0.01

In [6]:
img_preprocessor = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5, 1.5],
    channel_shift_range=10,
    vertical_flip=True,
    validation_split=0.2,
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=True,   # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,   # divide each input by its std
)

train_data_gen = img_preprocessor.flow_from_directory(FOLDER_PATH, target_size=TARGET_SIZE,
                                                      batch_size=BS, subset='training',
                                                      class_mode='categorical', shuffle=True)

val_data_gen = img_preprocessor.flow_from_directory(FOLDER_PATH,  target_size=TARGET_SIZE,
                                                    batch_size=BS, subset='validation', 
                                                    class_mode='categorical', shuffle=True)

Found 1520 images belonging to 38 classes.
Found 342 images belonging to 38 classes.


In [7]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications import MobileNetV2

In [8]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=IMAGE_SIZE)

In [9]:
for layers in base_model.layers:
    layers.trainable = False

In [35]:

x= mobilenetV2_model.output
x= GlobalAveragePooling2D()(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.2)(x)
prediction= Dense(38, activation = 'softmax')(x)
model= Model(inputs= mobilenetV2_model.input, outputs= prediction)

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [36]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    restore_best_weights=True  
)

In [37]:
epochs = 5
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps = len(val_data_gen)//BS,
    steps_per_epoch = len(train_data_gen)//BS,
    batch_size = BS,
    callbacks=[early_stopping]
    # class_weight=class_weights
)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 243s 19s/step - accuracy: 0.0469 - loss: 4.3533 - val_accuracy: 0.0000e+00 - val_loss: 7.2003
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.3224 - loss: 2.8132 - val_accuracy: 0.0000e+00 - val_loss: 6.6987
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.2132 - loss: 2.7179 - val_accuracy: 0.1875 - val_loss: 6.6123
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.3392 - loss: 2.5857 - val_accuracy: 0.0000e+00 - val_loss: 13.3064
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.3448 - loss: 2.4598 - val_accuracy: 0.0000e+00 - val_loss: 9.6003


In [34]:
mobilenetV2_model= tf.keras.applications.MobileNetV2(weights='imagenet',classes=38,include_top=False, input_shape=(224,224, 3))
x= mobilenetV2_model.output
x= GlobalAveragePooling2D()(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.2)(x)
prediction= Dense(38, activation = 'softmax')(x)
model= Model(inputs= mobilenetV2_model.input, outputs= prediction)

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


history = model.fit(x = train_data_gen, validation_data = val_data_gen, 
                    validation_steps = len(val_data_gen)//BS,
                    steps_per_epoch = len(train_data_gen)//BS,
                    batch_size = BS,
                    epochs = 20)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 88s 4s/step - accuracy: 0.0510 - loss: 4.0867 - val_accuracy: 0.1250 - val_loss: 3.5911
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.1891 - loss: 3.4697 - val_accuracy: 0.0625 - val_loss: 4.7314
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.1878 - loss: 3.3074 - val_accuracy: 0.0000e+00 - val_loss: 6.0594
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.2833 - loss: 2.9138 - val_accuracy: 0.0000e+00 - val_loss: 8.9402
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.2561 - loss: 2.9545 - val_accuracy: 0.0000e+00 - val_loss: 11.7441
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3241 - loss: 3.1249 - val_accuracy: 0.1875 - val_loss: 11.2240
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3625 - loss: 2.4103 - val_accuracy: 0.0000e+00 - val_loss: 12.3511
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.4550 - loss: 1.9574 - val_accuracy: 0.0625 - val_loss: 13.

C:\Users\Aathi\anaconda3\envs\ML\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 448ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 10.4855


In [9]:
from src.constants import *
from src.utils.utils import *
from src.config.entity import * 

class DataTrainingManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
    
    def get_parameters(self):
        config = self.config.trainingConfig
        

'd:\\FarmconProject\\plantClassificationPart'